# Objective

This experiment checks the following for a simple 2 layer FC network on MNIST.
1. Verify Agop and NFM relations for the conv layers
2. Run RFM to construct similar matrices as the above.(TBD)

The model is taken from MNIST/model3

# Setup

In [1]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir1= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model1\\nn_models\\'
model_dir3= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model3\\nn_models\\'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_fc as af
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import random
import torch.backends.cudnn as cudnn
from trained_models.MNIST.model1 import trainer as t1
from trained_models.MNIST.model3 import trainer as t3
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.nn as nn
from copy import deepcopy

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")

Using device: cuda:0


In [7]:
torch.cuda.empty_cache()

In [13]:
trainloader, valloader, testloader = t1.get_loaders()
net= t1.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir1+'mnist_fc_trained_nn.pth'):
    checkpoint = torch.load(model_dir1+'mnist_fc_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
t1.train_net()

torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
Train Size:  40000 Val Size:  10000
Test Size:  10000
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([892, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
Model weights loaded successfully.
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
Train Size:  40000 Va

In [ ]:
trainloader, valloader, testloader = t3.get_loaders()
net= t3.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir3+'mnist_fc_trained_nn.pth'):
    checkpoint = torch.load(model_dir3+'mnist_fc_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")
else:
    print("Train the network first")

# Verify NFA for FC layers:

In [29]:

print(net)
af.verify_NFA(net, init_net, trainloader, max_batch= 50, classes=10, chunk_idx=1, layer_idx=0)

Net(
  (features): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=False)
    (1): Nonlinearity()
    (2): Linear(in_features=1024, out_features=1024, bias=False)
    (3): Nonlinearity()
  )
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=10, bias=False)
  )
)
Correlation between Initial and Trained CNFM:  tensor(0.2483, device='cuda:0')
Computing GOP for sample 0 out of 50
Computing GOP for sample 1 out of 50
Computing GOP for sample 2 out of 50
Computing GOP for sample 3 out of 50
Computing GOP for sample 4 out of 50
Computing GOP for sample 5 out of 50
Computing GOP for sample 6 out of 50
Computing GOP for sample 7 out of 50
Computing GOP for sample 8 out of 50
Computing GOP for sample 9 out of 50
Computing GOP for sample 10 out of 50
Computing GOP for sample 11 out of 50
Computing GOP for sample 12 out of 50
Computing GOP for sample 13 out of 50
Computing GOP for sample 14 out of 50
Computing GOP for sample 15 out of 50
Computing GOP

tensor([[ 3.1685e-02, -9.7748e-04,  4.9163e-05,  ..., -2.1445e-03,
          1.2241e-03,  9.9545e-04],
        [-9.8003e-04,  2.9947e-02, -5.2103e-04,  ...,  1.9511e-05,
          5.8741e-04, -2.3383e-03],
        [ 5.0554e-05, -5.2306e-04,  3.0615e-02,  ..., -1.9127e-03,
          6.6194e-04,  2.1060e-04],
        ...,
        [-2.1529e-03,  2.3456e-05, -1.9159e-03,  ...,  3.0967e-02,
         -3.3085e-03, -2.0154e-04],
        [ 1.2234e-03,  5.8500e-04,  6.6632e-04,  ..., -3.3068e-03,
          3.3514e-02,  2.6245e-03],
        [ 9.9287e-04, -2.3378e-03,  2.1906e-04,  ..., -2.0068e-04,
          2.6247e-03,  3.0006e-02]], device='cuda:0')

In [ ]:
#TODO: How to meaningfully visualise? 

# RFM 

In [ ]:
'''Warning: This is an extremely cpu intensive process since it uses solve function from linalg 
The rfm.py from utils is equipped with more memory efficient solvers. 
'''

rfm.rfm(trainloader, valloader, testloader, name=name,
            batch_size=10, iters=1, reg=1e-3)